In [4]:
import lib_non_local_games as nlg
import numpy as np
import cvxpy as cp
import functools as fc
from operator import mul

In [5]:
cp.installed_solvers()

['CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'MOSEK', 'OSQP', 'SCS']

# CHSH game

In [6]:
dimA1 = 2
dimA2 = 2
dimQ1 = 3
dimQ2 = 3

dimT = 2
dimS = 2

V = [0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1]
probQ1 = [0.47442417, 0.004321,   0.52125483]
probQ2 = [0.43628622, 0.0444913,  0.51922248]

Parameter of the CHSH game

In [7]:
dimA1 = 2
dimA2 = 2
dimQ1 = 2
dimQ2 = 2

dimT = 2
dimS = 2

probQ1 = (.5,.5)
probQ2 = (.5,.5)

Initialisation

In [8]:
# Subsystems A1 Q1 A2 Q2
subs_A1Q1A2Q2 = (dimA1,dimQ1,dimA2,dimQ2)
indices_A1Q1A2Q2 = nlg.indices_list(subs_A1Q1A2Q2)
dim_A1Q1A2Q2 = fc.reduce(mul, subs_A1Q1A2Q2, 1)

# Subsystems A2 Q1 Q2
subs_A2Q1Q2 = (dimA2,dimQ1,dimQ2)
indices_A2Q1Q2 = nlg.indices_list(subs_A2Q1Q2)

# Subsystems A1 Q1 Q2
subs_A1Q1Q2 = (dimA1,dimQ1,dimQ2)
indices_A1Q1Q2 = nlg.indices_list(subs_A1Q1Q2)

# Subsystems A1 Q1
subs_A1Q1 = (dimA1,dimQ1)
indices_A1Q1 = nlg.indices_list(subs_A1Q1)
dim_A1Q1 = fc.reduce(mul, subs_A1Q1, 1)

# Subsystems A2 Q2
subs_A2Q2 = (dimA2,dimQ2)
indices_A2Q2 = nlg.indices_list(subs_A2Q2)
dim_A2Q2 = fc.reduce(mul, subs_A2Q2, 1)

# Subsystems A1 A2
subs_A1A2 = (dimA1,dimA2)
indices_A1A2 = nlg.indices_list(subs_A1A2)

# Subsystems Q1 Q2
subs_Q1Q2 = (dimQ1,dimQ2)
indices_Q1Q2 = nlg.indices_list(subs_Q1Q2)

# Subsystems T hat(T) S hat(S)
subs_TTSS = (dimT,dimT,dimS,dimS)
indices_TTSS = nlg.indices_list(subs_TTSS)
dim_TTSS = fc.reduce(mul, subs_TTSS, 1)

# Additional dimensions
dim_TSS = dimT * dimS**2
dim_TT = dimT**2
dim_SS = dimS**2

# State on subsystem T
rhoT = np.identity(dimT)/dimT

## CHSH classical value

Compute the following level of the hierarchy

In [9]:
level = 1

The program

In [10]:
## VARIABLE AND CONSTRAINTS
constraints = []

# Call the function which generates the variable and the constraints 
classical_prob, BtI_ext = nlg.classical_constraints(constraints,subs_A1Q1,subs_A2Q2,probQ1,probQ2,level)

## OBJECTIVE FUNCTION

# The rule function is
rule_A1Q1A2Q2 = nlg.CHSH_rule_function_A1Q1A2Q2
indices_A2Q2_ext_butone = nlg.indices_list(subs_A2Q2*(level-1))
object_function = 0

# The object function is
for index_A1Q1A2Q2 in indices_A1Q1A2Q2:
    indices_a1q1a2q2_ext = nlg.fuse_arrays(np.array([index_A1Q1A2Q2]),indices_A2Q2_ext_butone)
    object_function += rule_A1Q1A2Q2(*index_A1Q1A2Q2) * sum([classical_prob[BtI_ext(i)] for i in indices_a1q1a2q2_ext])
    
# Write the problem
prob = cp.Problem(cp.Maximize(object_function), constraints)

Solve the program

In [11]:
prob.solve(verbose=True,solver='MOSEK')



Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 33              
  Cones                  : 0               
  Scalar variables       : 16              
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 33              
  Cones                  : 0               
  Scalar variables       : 16              
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 8
Optimizer  - Cones                  : 0
Optimizer  - Scalar variables      

1.0000000010619423

## CHSH $SDP_{n_1 = 1, n_2 = 1}$ with NPA constraints

Further initialisation

In [5]:
# Classical level
class_level = 2

# Indices for the extension of A2Q2
indices_A2Q2_ext_butone = nlg.indices_list(subs_A2Q2*(class_level-1))

# Function for mapping indices to integers
BtI = lambda seq : nlg.seqtoint(seq, subs_A1Q1A2Q2)

# Maximally entangled vectors between T|S and TT|SS
phi_TS = nlg.bipartite_unnorm_max_entangled_state(dimT)
phi_TSTS = nlg.tensor([phi_TS,phi_TS])

# Maximally mixed states between TT|SS (correct order of subsystems)
Phi_TSTS = np.outer(phi_TSTS,phi_TSTS)
P = nlg.permutation_matrix((0,1,2,3), (0,2,1,3), (dimT, dimT,dimT, dimT))
Phi_TTSS = P @ Phi_TSTS @ P

The program

In [8]:
## VARIABLES

# The (sub-normalized) states we optimize over
rho_TTSS = []
for i in map(BtI,indices_A1Q1A2Q2):
    rho_TTSS.append( cp.Variable((dim_TTSS,dim_TTSS),hermitian=True) )

## OBJECTIVE FUNCTION

# The swap operator is
F_TTSS = nlg.permutation_matrix((0,1), (1,0), (dim_TT, dim_SS))

# The rule function is
#rule_A1Q1A2Q2 = nlg.CHSH_rule_function_A1Q1A2Q2
#rule_A1Q1A2Q2, V = nlg.generate_random_rule_function_A1Q1A2Q2(subs_A1Q1A2Q2)
rule_A1Q1A2Q2 = nlg.generate_rule_function_from_array(V,subs_A1Q1A2Q2)

# The object function is
obj_fun_components = [rule_A1Q1A2Q2(*index) * cp.trace( cp.matmul(Phi_TTSS,rho_TTSS[BtI(index)]) )
                      for index in indices_A1Q1A2Q2]

object_function = cp.Constant(dimT**2) * sum(obj_fun_components)

## CONSTRAINTS

constraints = []
    
# 2) rho_TTSS are (sub-normalized) quantum states
# 2a) trace of the sum is 1
constraints.append( sum([cp.trace(rho_TTSS[i]) for i in map(BtI,indices_A1Q1A2Q2)]) - 1 == 0 )

# 2b) positive semidefinite matrices
for i in map(BtI,indices_A1Q1A2Q2):
    constraints.append( rho_TTSS[i] >> 0 )
    
# 3) First linear constraint
for a2,q1,q2 in indices_A2Q1Q2:
    indices_A1q1a2q2 = [BtI([a,q1,a2,q2]) for a in range(dimA1)]
    indices_A1Q1a2q2 = [BtI([a,q,a2,q2]) for a,q in indices_A1Q1]
    
    lhs = sum([rho_TTSS[i] for i in indices_A1q1a2q2])
    
    rhs_variable = sum([rho_TTSS[i] for i in indices_A1Q1a2q2])
    rhs_partial = nlg.partial_trace(rhs_variable, [dimT, dim_TSS])
    rhs = cp.Constant(probQ1[q1]) * cp.kron(rhoT, rhs_partial)
    
    constraints.append( lhs - rhs == 0 )
    
# 4) Second linear constraint
P = nlg.permutation_matrix((0,1,2,3),(1,0,2,3),subs_TTSS)

for a1,q1,q2 in indices_A1Q1Q2:
    indices_a1q1A2q2 = [BtI([a1,q1,a,q2]) for a in range(dimA2)]
    indices_a1q1A2Q2 = [BtI([a1,q1,a,q]) for a,q in indices_A2Q2]
    
    lhs_variable = sum([rho_TTSS[i] for i in indices_a1q1A2q2])
    lhs = cp.matmul( cp.matmul(P,lhs_variable) , P )
    
    rhs_variable = sum([rho_TTSS[i] for i in indices_a1q1A2Q2])
    rhs_permuted = cp.matmul( cp.matmul(P,rhs_variable) , P )
    rhs_partial = nlg.partial_trace(rhs_permuted, [dimT, dim_TSS])
    rhs = cp.Constant(probQ2[q2]) * cp.kron(rhoT, rhs_partial)
    
    constraints.append( lhs - rhs == 0 )

# 5) Assuming the operators are projectors

# 5.1) Tracing TT gives constant independent of a1,a2
for a1,q1,a2,q2 in indices_A1Q1A2Q2:
    rhs = nlg.partial_trace(rho_TTSS[BtI([a1,q1,a2,q2])], [dim_TT, dim_SS])/(probQ1[q1]*probQ2[q2])
    for a1p,q1p,a2p,q2p in indices_A1Q1A2Q2:
        lhs = nlg.partial_trace(rho_TTSS[BtI([a1p,q1p,a2p,q2p])], [dim_TT, dim_SS])/(probQ1[q1p]*probQ2[q2p])
        constraints.append( rhs - lhs == 0 )
    
# 5.2) Tracing T gives constant independent of a1,q1
for a1,q1,a2,q2 in indices_A1Q1A2Q2:
    rhs = nlg.partial_trace(rho_TTSS[BtI([a1,q1,a2,q2])], [dimT, dim_TSS])/probQ1[q1]
    for a1p,q1p in indices_A1Q1:
        lhs = nlg.partial_trace(rho_TTSS[BtI([a1p,q1p,a2,q2])], [dimT, dim_TSS])/probQ1[q1p]
        constraints.append( rhs - lhs == 0 )
        
# 5.3) Tracing hat{T} gives constant independent of a2,q2
P = nlg.permutation_matrix((0,1,2,3),(1,0,2,3),subs_TTSS)
for a1,q1,a2,q2 in indices_A1Q1A2Q2:
    rhs_variable = cp.matmul( cp.matmul(P,rho_TTSS[BtI([a1,q1,a2,q2])]) , P )
    rhs = nlg.partial_trace(rhs_variable, [dimT, dim_TSS])/probQ2[q2]
    for a2p,q2p in indices_A2Q2:
        lhs_variable = cp.matmul( cp.matmul(P,rho_TTSS[BtI([a1,q1,a2p,q2p])]) , P )
        lhs = nlg.partial_trace(lhs_variable, [dimT, dim_TSS])/probQ2[q2p]
        constraints.append( rhs - lhs == 0 )

# # 6) Classical distribution

# # Add the classical constraints
# classical_prob, BtI_ext = nlg.classical_constraints(constraints,subs_A1Q1,subs_A2Q2,probQ1,probQ2,class_level)

# # Connect the matrix variable with the probability distribution
# for index_A1Q1A2Q2 in indices_A1Q1A2Q2:
#     indices_a1q1a2q2_ext = nlg.fuse_arrays(np.array([index_A1Q1A2Q2]),indices_A2Q2_ext_butone)
    
#     rhs = cp.trace(rho_TTSS[BtI(index_A1Q1A2Q2)])
#     lhs = sum([classical_prob[BtI_ext(i)] for i in indices_a1q1a2q2_ext])
    
#     constraints.append( rhs - lhs == 0 )

# # 7) NPA style constraint (see PhysRevLett.98.010401)

# # Introduce normalization factor
# renorm = lambda x,y : dimT**2/(probQ1[x]*probQ2[y])

# # The P matrix containing the information about the variables rho_TTSS is give by
# P = []

# for a1,q1 in indices_A1Q1:
#     P_row = [renorm(q1,q2)*cp.trace(Phi_TTSS @ rho_TTSS[BtI([a1,q1,a2,q2])]) for a2,q2 in indices_A2Q2]
#     P.append(P_row)

# P = cp.bmat(P)

# # The Q matrix containing information on the variables rho_TTSS and new variables as well
# Q = []

# for a1,q1 in indices_A1Q1:
#     Q_row = []
#     for a1p,q1p in indices_A1Q1:
#         if q1 == q1p:
#             if a1 == a1p:
#                 val = sum([renorm(q1,q2)*cp.trace(Phi_TTSS@rho_TTSS[BtI([a1,q1,a2,q2])]) for a2,q2 in indices_A2Q2])/dimQ2
#             else:
#                 val = cp.Constant(0)
#         else:
#             val = cp.Variable()
#         Q_row.append(val)
#     Q.append(Q_row)

# Q = cp.bmat(Q)

# # The R matrix containing information on the variables rho_TTSS and new variables as well
# R = []

# for a2,q2 in indices_A2Q2:
#     R_row = []
#     for a2p,q2p in indices_A2Q2:
#         if q2 == q2p:
#             if a2 == a2p:
#                 val = sum([renorm(q1,q2)*cp.trace(Phi_TTSS@rho_TTSS[BtI([a1,q1,a2,q2])]) for a1,q1 in indices_A1Q1])/dimQ1
#             else:
#                 val = cp.Constant(0)
#         else:
#             val = cp.Variable()
#         R_row.append(val)
#     R.append(R_row)

# R = cp.bmat(R)

# # Constructing vector v of dimension dimA1*dimQ1+dimA2*dimQ2
# v = []

# for a1,q1 in indices_A1Q1:
#     v.append(sum([renorm(q1,q2)*cp.trace(Phi_TTSS @ rho_TTSS[BtI([a1,q1,a2,q2])]) for a2,q2 in indices_A2Q2])/dimQ2)
    
# for a2,q2 in indices_A2Q2:
#     v.append(sum([renorm(q1,q2)*cp.trace(Phi_TTSS @ rho_TTSS[BtI([a1,q1,a2,q2])]) for a1,q1 in indices_A1Q1])/dimQ1)
    
# v = cp.bmat([v])
# w = cp.vstack([cp.Constant([[1]]),v.T])

# # Builiding the matrix M that should be positive semi-definite (NPA constraint)
# M = cp.vstack([cp.hstack([Q,P]),cp.hstack([P.T,R])])
# M = cp.vstack([v,M])
# M = cp.hstack([w,M])
      
# constraints.append( M >> 0 )
# constraints.append( M - M.T == 0 )
        
# 8) PPT criterium
for i in map(BtI,indices_A1Q1A2Q2):
    constraints.append( nlg.partial_transpose(rho_TTSS[i],subs_TTSS,(0,0,1,1)) >> 0 )
    constraints.append( nlg.partial_transpose(rho_TTSS[i],subs_TTSS,(1,0,0,0)) >> 0 )
    constraints.append( nlg.partial_transpose(rho_TTSS[i],subs_TTSS,(0,1,0,0)) >> 0 )
    
# Write the problem
prob = cp.Problem(cp.Maximize(cp.real(object_function)), constraints)

Solve the problem

In [9]:
prob.solve(verbose=True,solver='MOSEK')



Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 262658          
  Cones                  : 0               
  Scalar variables       : 14112           
  Matrix variables       : 144             
  Integer variables      : 0               

Optimizer started.
GP based matrix reordering started.
GP based matrix reordering terminated.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 262658          
  Cones                  : 0               
  Scalar variables       : 14112           
  Matrix variables       : 144             
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            

0.9975648812286765

In [ ]:
0.9975642076731657

A different constraint, used in Application I of the PRL paper

In [ ]:
# The P matrix containing the information about the variables rho_TTSS is give by
P = []

for q1 in range(dimQ1):
    P_row = []
    for q2 in range(dimQ2):
        # Normalization constant to get P(a1,a2|q1,q2)
        norm = cp.Constant( dimT**2/(probQ1[q1]*probQ2[q2]) )
        # Compute the correlation function C_{q1,q2} = sum_{a1,a2} (-1)^{a1*a2} p(a1,a2|q1,q2)
        C = sum([(-1)**(a1+a2)*norm*cp.trace(cp.matmul(Phi_TTSS,rho_TTSS[BtI([a1,q1,a2,q2])])) for a1,a2 in indices_A1A2])
        P_row.append(C)
    P.append(P_row)

P = cp.bmat(P)

# Introduce new variables Q and R, symmetic matrices whose elements are < 1

# Q is dimQ1 x dimQ1
Q = cp.Variable((dimQ1,dimQ1),symmetric=True)
# R is dimQ2 x dimQ2
R = cp.Variable((dimQ2,dimQ2),symmetric=True)

# 5.1) The matrix M = [[Q,P],[P^T,R]] is PSD (NPA constraint)
M = cp.bmat([[Q,P],[P.T,R]])
constraints.append( M >> 0 )

# 5.2) Assumptions on projective nature of measurement on A and B
# Assumption over Q
for q in range(dimQ1):
    constraints.append( Q[q,q] - 1 == 0 )
    
# Assumption over R
for q in range(dimQ2):
    constraints.append( R[q,q] - 1 == 0 )

# I3322 game ($n_1 = 1$, $n_2 = 1$) with NPA constraints

NOTE: The $I_{3322}$ is given in terms of conditional probabilities, while our games is not. For this specific setting, we can unify the two by multiplying our result by 9, that is, the inverse of $\pi(q_1)\pi(q_2)$ for all $q_1, q_2$, since these distributions are uniform.

In [15]:
dimA1 = 2
dimA2 = 2
dimQ1 = 3
dimQ2 = 3

dimT = 2
dimS = 2

probQ1 = (1./3,1./3,1./3)
probQ2 = (1./3,1./3,1./3)

eta = 1.

In [16]:
# Subsystems A1 Q1 A2 Q2
subs_A1Q1A2Q2 = (dimA1,dimQ1,dimA2,dimQ2)
indices_A1Q1A2Q2 = nlg.indices_list(subs_A1Q1A2Q2)
dim_A1Q1A2Q2 = fc.reduce(mul, subs_A1Q1A2Q2, 1)

# Subsystems A2 Q1 Q2
subs_A2Q1Q2 = (dimA2,dimQ1,dimQ2)
indices_A2Q1Q2 = nlg.indices_list(subs_A2Q1Q2)

# Subsystems A1 Q1 Q2
subs_A1Q1Q2 = (dimA1,dimQ1,dimQ2)
indices_A1Q1Q2 = nlg.indices_list(subs_A1Q1Q2)

# Subsystems A1 Q1
subs_A1Q1 = (dimA1,dimQ1)
indices_A1Q1 = nlg.indices_list(subs_A1Q1)

# Subsystems A2 Q2
subs_A2Q2 = (dimA2,dimQ2)
indices_A2Q2 = nlg.indices_list(subs_A2Q2)

# Subsystems T hat(T) S hat(S)
subs_TTSS = (dimT,dimT,dimS,dimS)
indices_TTSS = nlg.indices_list(subs_TTSS)
dim_TTSS = fc.reduce(mul, subs_TTSS, 1)

# Additional dimensions
dim_TSS = dimT * dimS**2
dim_TT = dimT**2
dim_SS = dimS**2

# State on subsystem T
rhoT = np.identity(dimT)/dimT

Rule function and list-to-integer function: 

In [17]:
def I3322_rule_function(a1,q1,a2,q2,eta=1.):
    
    index = str(a1)+str(a2)+str(q1)+str(q2)

    I3322_rules = {"0000":1./3-1./(3*eta),
                   "0001":2./3-1./(3*eta),
                   "0002":1.-1./(3*eta),
                   "0010":1./3,
                   "0011":2./3,
                   "0012":-1.,
                   "0020":1./3,
                   "0021":-4./3,
                   "0100":-1./(3*eta),
                   "0101":-1./(3*eta),
                   "0102":-1./(3*eta),
                   "1000":-2./3,
                   "1001":-1./3,
                   "1010":-2./3,
                   "1011":-1./3,
                   "1020":-2./3,
                   "1021":-1./3}
    
    try:
        out = I3322_rules[index]
    except KeyError:
        out = 0
    
    return out

# Here we define the function from sequence to integer for the I3322
StI = lambda seq : nlg.seqtoint(seq, subs_A1Q1A2Q2)

In [18]:
# Maximally entangled vectors between T|S and TT|SS
phi_TS = nlg.bipartite_unnorm_max_entangled_state(dimT)
phi_TSTS = nlg.tensor([phi_TS,phi_TS])

# Maximally mixed states between TT|SS (correct order of subsystems)
Phi_TSTS = np.outer(phi_TSTS,phi_TSTS)
P = nlg.permutation_matrix((0,1,2,3), (0,2,1,3), (dimT, dimT,dimT, dimT))
Phi_TTSS = P @ Phi_TSTS @ P

The program:

In [25]:
## VARIABLES

# The (sub-normalized) states we optimize over
rho_TTSS = []
for i in map(StI,indices_A1Q1A2Q2):
    rho_TTSS.append( cp.Variable((dim_TTSS,dim_TTSS),hermitian=True) )

## OBJECTIVE FUNCTION

# The rule function is
rule_A1Q1A2Q2 = I3322_rule_function

# The object function is
obj_fun_components = [rule_A1Q1A2Q2(*index) * cp.trace( cp.matmul(Phi_TTSS,rho_TTSS[StI(index)]) )
                      for index in indices_A1Q1A2Q2]

object_function = cp.Constant(dimT**2) * sum(obj_fun_components)

## CONSTRAINTS

constraints = []
    
# 2) rho_TTSS are (sub-normalized) quantum states
# 2a) trace of the sum is 1
constraints.append( sum([cp.trace(rho_TTSS[i]) for i in map(StI,indices_A1Q1A2Q2)]) - 1 == 0 )

# 2b) positive semidefinite matrices
for i in map(StI,indices_A1Q1A2Q2):
    constraints.append( rho_TTSS[i] >> 0 )
    
# 3) First linear constraint
for a2,q1,q2 in indices_A2Q1Q2:
    indices_A1q1a2q2 = [StI([a,q1,a2,q2]) for a in range(dimA1)]
    indices_A1Q1a2q2 = [StI([a,q,a2,q2]) for a,q in indices_A1Q1]
    
    lhs = sum([rho_TTSS[i] for i in indices_A1q1a2q2])
    
    rhs_variable = sum([rho_TTSS[i] for i in indices_A1Q1a2q2])
    rhs_partial = nlg.partial_trace(rhs_variable, [dimT, dim_TSS])
    rhs = cp.Constant(probQ1[q1]) * cp.kron(rhoT, rhs_partial)
    
    constraints.append( lhs - rhs == 0 )
    
# 4) Second linear constraint
P = nlg.permutation_matrix((0,1,2,3),(1,0,2,3),subs_TTSS)

for a1,q1,q2 in indices_A1Q1Q2:
    indices_a1q1A2q2 = [StI([a1,q1,a,q2]) for a in range(dimA2)]
    indices_a1q1A2Q2 = [StI([a1,q1,a,q]) for a,q in indices_A2Q2]
    
    lhs_variable = sum([rho_TTSS[i] for i in indices_a1q1A2q2])
    lhs = cp.matmul( cp.matmul(P,lhs_variable) , P )
    
    rhs_variable = sum([rho_TTSS[i] for i in indices_a1q1A2Q2])
    rhs_permuted = cp.matmul( cp.matmul(P,rhs_variable) , P )
    rhs_partial = nlg.partial_trace(rhs_permuted, [dimT, dim_TSS])
    rhs = cp.Constant(probQ2[q2]) * cp.kron(rhoT, rhs_partial)
    
    constraints.append( lhs - rhs == 0 )

# 5) Assuming the operators are projectors

# 5.1) Tracing TT gives constant independent of a1,a2
for a1,q1,a2,q2 in indices_A1Q1A2Q2:
    rhs = nlg.partial_trace(rho_TTSS[BtI([a1,q1,a2,q2])], [dim_TT, dim_SS])/(probQ1[q1]*probQ2[q2])
    for a1p,q1p,a2p,q2p in indices_A1Q1A2Q2:
        lhs = nlg.partial_trace(rho_TTSS[BtI([a1p,q1p,a2p,q2p])], [dim_TT, dim_SS])/(probQ1[q1p]*probQ2[q2p])
        constraints.append( rhs - lhs == 0 )
    
# 5.2) Tracing T gives constant independent of a1,q1
for a1,q1,a2,q2 in indices_A1Q1A2Q2:
    rhs = nlg.partial_trace(rho_TTSS[BtI([a1,q1,a2,q2])], [dimT, dim_TSS])/probQ1[q1]
    for a1p,q1p in indices_A1Q1:
        lhs = nlg.partial_trace(rho_TTSS[BtI([a1p,q1p,a2,q2])], [dimT, dim_TSS])/probQ1[q1p]
        constraints.append( rhs - lhs == 0 )
        
# 5.3) Tracing hat{T} gives constant independent of a2,q2
P = nlg.permutation_matrix((0,1,2,3),(1,0,2,3),subs_TTSS)
for a1,q1,a2,q2 in indices_A1Q1A2Q2:
    rhs_variable = cp.matmul( cp.matmul(P,rho_TTSS[BtI([a1,q1,a2,q2])]) , P )
    rhs = nlg.partial_trace(rhs_variable, [dimT, dim_TSS])/probQ2[q2]
    for a2p,q2p in indices_A2Q2:
        lhs_variable = cp.matmul( cp.matmul(P,rho_TTSS[BtI([a1,q1,a2p,q2p])]) , P )
        lhs = nlg.partial_trace(lhs_variable, [dimT, dim_TSS])/probQ2[q2p]
        constraints.append( rhs - lhs == 0 )    
    
# 6) NPA style constraint (see PhysRevLett.98.010401)

# Introduce normalization factor
renorm = lambda x,y : dimT**2/(probQ1[x]*probQ2[y])

# The P matrix containing the information about the variables rho_TTSS is give by
P = []

for a1,q1 in indices_A1Q1:
    P_row = [renorm(q1,q2)*cp.trace(Phi_TTSS @ rho_TTSS[StI([a1,q1,a2,q2])]) for a2,q2 in indices_A2Q2]
    P.append(P_row)

P = cp.bmat(P)

# The Q matrix containing information on the variables rho_TTSS and new variables as well
Q = []

for a1,q1 in indices_A1Q1:
    Q_row = []
    for a1p,q1p in indices_A1Q1:
        if q1 == q1p:
            if a1 == a1p:
                val = sum([renorm(q1,q2)*cp.trace(Phi_TTSS@rho_TTSS[StI([a1,q1,a2,q2])]) for a2,q2 in indices_A2Q2])/dimQ2
            else:
                val = cp.Constant(0)
        else:
            val = cp.Variable()
        Q_row.append(val)
    Q.append(Q_row)

Q = cp.bmat(Q)

# The R matrix containing information on the variables rho_TTSS and new variables as well
R = []

for a2,q2 in indices_A2Q2:
    R_row = []
    for a2p,q2p in indices_A2Q2:
        if q2 == q2p:
            if a2 == a2p:
                val = sum([renorm(q1,q2)*cp.trace(Phi_TTSS@rho_TTSS[StI([a1,q1,a2,q2])]) for a1,q1 in indices_A1Q1])/dimQ1
            else:
                val = cp.Constant(0)
        else:
            val = cp.Variable()
        R_row.append(val)
    R.append(R_row)

R = cp.bmat(R)

# Constructing vector v of dimension dimA1*dimQ1+dimA2*dimQ2
v = []

for a1,q1 in indices_A1Q1:
    v.append(sum([renorm(q1,q2)*cp.trace(Phi_TTSS @ rho_TTSS[StI([a1,q1,a2,q2])]) for a2,q2 in indices_A2Q2])/dimQ2)
    
for a2,q2 in indices_A2Q2:
    v.append(sum([renorm(q1,q2)*cp.trace(Phi_TTSS @ rho_TTSS[StI([a1,q1,a2,q2])]) for a1,q1 in indices_A1Q1])/dimQ1)
    
v = cp.bmat([v])
w = cp.vstack([cp.Constant([[1]]),v.T])

# Builiding the matrix M that should be positive semi-definite (NPA constraint)
M = cp.vstack([cp.hstack([Q,P]),cp.hstack([P.T,R])])
M = cp.vstack([v,M])
M = cp.hstack([w,M])
      
constraints.append( M >> 0 )
constraints.append( M - M.T == 0 )

# 7) PPT criterium
for i in map(StI,indices_A1Q1A2Q2):
    constraints.append( nlg.partial_transpose(rho_TTSS[i],subs_TTSS,(0,0,1,1)) >> 0 )
    constraints.append( nlg.partial_transpose(rho_TTSS[i],subs_TTSS,(1,0,0,0)) >> 0 )
    constraints.append( nlg.partial_transpose(rho_TTSS[i],subs_TTSS,(0,1,0,0)) >> 0 )
    
# Write the problem
prob = cp.Problem(cp.Maximize(cp.real(object_function)), constraints)

In [26]:
prob.solve(verbose=True,solver='MOSEK')



Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 165890          
  Cones                  : 0               
  Scalar variables       : 14112           
  Matrix variables       : 144             
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 165890          
  Cones                  : 0               
  Scalar variables       : 14112           
  Matrix variables       : 144             
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 149937
Optimizer  - Cones                  : 1
Optimizer  - Scalar variab

0.1111111114272956

The $I_{3322}$ value for $\eta = 1$

In [ ]:
I3322 = 9 * prob.value
print(I3322)